In [2]:
import gspread
from gspread_dataframe import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials

def upload_df_to_gsheet(df, tab_name, creds_path, sheet_title, start_cell="A1"):
    scope = ['https://spreadsheets.google.com/feeds',
             'https://www.googleapis.com/auth/drive']

    creds = ServiceAccountCredentials.from_json_keyfile_name(creds_path, scope)
    client = gspread.authorize(creds)

    sheet = client.open(sheet_title)
    worksheet = sheet.worksheet(tab_name)

    # Clear the sheet starting from A1, optional
    # worksheet.clear()

    # Upload with formulas being parsed correctly
    set_with_dataframe(worksheet, df, row=1, col=1, include_index=False, include_column_header=True, resize=True)


In [3]:
def add_zillow_link_column(df):
    def build_zillow_url(row):
        # Adjust based on your actual column names
        address = str(row.get("address", "")).strip().replace(" ", "-").replace(".", "")
        city = str(row.get("city", "Las Vegas")).strip().replace(" ", "-")
        state = str(row.get("state", "NV")).strip()
        zip_code = str(row.get("zip", "")).strip()

        components = [address, city, state, zip_code]
        url_slug = "-".join([c for c in components if c])  # drop empty parts

        return f'=HYPERLINK("https://www.zillow.com/homes/{url_slug}_rb/", "View")'

    df["Zillow Link"] = df.apply(build_zillow_url, axis=1)
    return df

In [ ]:


# ✅ 2. Define GSheet upload parameters
title = "Lead Generation Tool"   # ← This is the title of the Google Sheet
tab = "Sheet1"                   # ← This is the name of the tab inside the sheet
creds_path = "api_access.json"

# ✅ 3. Upload
upload_df_to_gsheet(df, tab, creds_path, title)
print("✅ Upload complete.")

In [ ]:
# -------------------------------
# export/export_singlefamily.py
# -------------------------------

from etl.backup_loader import run_query

# from gsheet import upload_df_to_gsheet  # <- adjust if path differs

import pandas as pd

# -------------------------------
# 1. Define the Query
# -------------------------------
query = """
SELECT
  *
FROM analytics.ana_single_prop
ORDER BY total_score DESC
"""

df = run_query(query)

# -------------------------------
# 2. Format for Readability
# -------------------------------
import datetime

def clean_export_dataframe(df):
    df = df.copy()

    # Format currency columns with commas
    if 'mls_amount' in df.columns:
        df['mls_amount'] = df['mls_amount'].round().astype('Int64').map("{:,}".format)

    if 'price_per_sqft' in df.columns:
        df['price_per_sqft'] = df['price_per_sqft'].round().astype('Int64').map("{:,}".format)

    if 'building_sqft' in df.columns:
        df['building_sqft'] = df['building_sqft'].round().astype('Int64').map("{:,}".format)

    # Round score columns
    score_cols = ['price_score', 'equity_score', 'mls_days_score', 'hoa_penalty', 'total_score']
    for col in score_cols:
        if col in df.columns:
            df[col] = df[col].round(1)

    # Convert datetime/date columns to strings
    for col in df.columns:
        if pd.api.types.is_datetime64_any_dtype(df[col]) or df[col].apply(lambda x: isinstance(x, (datetime.date, pd.Timestamp))).any():
            df[col] = df[col].astype(str)

    return df
    

df_cleaned = clean_export_dataframe(df)
df_cleaned = add_zillow_link_column(df_cleaned)

desired_order = [
    "Zillow Link",     
    "city",      # move link to first column
    "address", 
     
    "state", 
    "zip_code",
    "mls_amount", 
    "price_per_sqft", 
    "building_sqft", 
    "price_score", 
    "equity_score", 
    "mls_days_score", 
    "hoa_penalty", 
    "total_score", 
    "extract_date"
]

df_cleaned = df_cleaned[desired_order]

# -------------------------------
# 3. Upload to Google Sheets
# -------------------------------
sheet_title = "Lead Generation Tool"
tab_name = "Single Family Leads_1"

creds_path = "api_access.json"

cols = df_cleaned.columns.tolist()
if "Zillow Link" in cols:
    cols.insert(0, cols.pop(cols.index("Zillow Link")))
    df_cleaned = df_cleaned[cols]

upload_df_to_gsheet(df_cleaned, tab_name, creds_path, sheet_title, "A1")
print("✅ Export complete.")

In [ ]:
# -------------------------------
# export/export_singlefamily.py
# -------------------------------

from etl.backup_loader import run_query
import pandas as pd
import datetime

# -------------------------------
# 1. Define the Query
# -------------------------------
query = """
SELECT
  *
FROM analytics.ana_single_prop
ORDER BY total_score DESC
"""
df = run_query(query)

# -------------------------------
# 2. Define Cleaning Function
# -------------------------------
def clean_export_dataframe(df):
    df = df.copy()

    # Format currency columns with commas
    for col in ['mls_amount', 'price_per_sqft', 'building_sqft']:
        if col in df.columns:
            df[col] = df[col].round().astype('Int64').map("{:,}".format)

    # Round score columns if present
    score_cols = ['total_score']
    for col in score_cols:
        if col in df.columns:
            df[col] = df[col].round(1)

    # Convert date/datetime to string
    for col in df.columns:
        if pd.api.types.is_datetime64_any_dtype(df[col]) or df[col].apply(lambda x: isinstance(x, (datetime.date, pd.Timestamp))).any():
            df[col] = df[col].astype(str)

    return df

# -------------------------------
# 3. Add Zillow Link Column
# -------------------------------
def add_zillow_link_column(df):
    if "address" in df.columns and "zip" in df.columns:
        def make_url(row):
            addr = row['address'].replace(" ", "-")
            zip_code = str(row['zip'])
            return f'=HYPERLINK("https://www.zillow.com/homes/{addr}-Las-Vegas-NV-{zip_code}_rb/", "View")'
        df.insert(0, "Zillow Link", df.apply(make_url, axis=1))
    return df

# -------------------------------
# 4. Clean and Reorder Columns
# -------------------------------
df_cleaned = clean_export_dataframe(df)
df_cleaned = add_zillow_link_column(df_cleaned)

desired_order = [
    "Zillow Link",
    "price_per_sqft",
    "mls_amount",
    "est_value",
    "diff",
    "perc_price_inc",
    "listed_price_inc",
    "total_score",
    "mls_days_on_market",
    "address",
    "building_sqft",
    "lot_size_sqft",
    "lot_coverage_ratio",
    "lot_size_per_building_sqft",
    "mls_date",
    "last_sale_date",
    "last_sale_amount",
    "total_open_loans",
    "total_loan_balance",
    "lien_amount",
    "est_equity_calc",
    "ltv_calc",
    "effective_year_built",
    "is_owner_occupied",
    "owner_1_first_name",
    "owner_1_last_name",
    "is_vacant",
    "total_condition",
    "mls_agent_name",
    "mls_agent_phone",
    "mls_agent_email",
    "mls_brokerage_name",
    "mls_brokerage_phone"
]

# Safely reorder columns
existing_columns = [col for col in desired_order if col in df_cleaned.columns]
df_cleaned = df_cleaned[existing_columns]

# -------------------------------
# 5. Upload to Google Sheets
# -------------------------------
sheet_title = "Lead Generation Tool"
tab_name = "Single Family Leads_1"
creds_path = "api_access.json"

upload_df_to_gsheet(df_cleaned, tab_name, creds_path, sheet_title, "A1")
print("✅ Export complete.")

✅ Export complete.
